In [ ]:
import tvm

import tvm.te as te

from tvm.script import ir as I
from tvm.script import tir as T
from tvm.script import relax as R


def showmod(mod: tvm.ir.module.IRModule):
    mod.show(
        black_format=True,
        show_meta=False,
        verbose_expr=True,
        show_object_address=False,
        show_all_struct_info=True,
    )


def createandshowmod(ops):
    te_func = te.create_prim_func(ops).with_attrs({"global_symbol": "test"})
    mod = tvm.IRModule({"test": te_func})
    showmod(mod)


from tvm.tir.op import *

#### _pack_buffer

In [ ]:
"""Build intrinsics that packs the buffer."""

from tvm.tir.op import _pack_buffer

buf = tvm.tir.decl_buffer(shape=(10, 10), dtype="float32")
showmod(_pack_buffer(buf))

#### call_packed_lowered

In [ ]:
"""Lowered version of call packed.
The argument to packed function can be Expr or Buffer.
The argument is the corresponding POD type when Expr is presented.
When the argument is Buffer, the corresponding PackedFunc
will recieve an TVMArrayHandle whose content is valid during the callback period.
If the PackedFunc is a python callback, then the corresponding argument is NDArray.

Parameters
----------
args : list of Expr or Buffer.
    Positional arguments.

span : Optional[Span]
    The location of this operator in the source code.

Returns
-------
call : PrimExpr
    The call expression.

See Also
--------
te.extern : Create tensor with extern function call.
"""

x: tvm.tir.PrimExpr = tvm.tir.IntImm("int32", 100)
y: tvm.tir.PrimExpr = tvm.tir.FloatImm("float32", 100)
showmod(call_packed_lowered(buf, x, y))

#### call_cpacked_lowered

In [ ]:
showmod(call_cpacked_lowered(buf, x, y))

#### call_packed

In [ ]:
showmod(call_packed(buf, x, y))

#### call_cpacked

In [ ]:
showmod(call_cpacked(buf, x, y))

#### call_intrin

In [ ]:
"""Build expression by calling an intrinsic function.

Intrinsics can be overloaded with multiple data types via
the intrinsic translation rule.

Parameters
----------
dtype : str
    The data type of the result.

func_name: str
    The intrinsic function name.

args : list
    Positional arguments.

span : Optional[Span]
    The location of this operator in the source code.

Returns
-------
call : PrimExpr
    The call expression.
"""

x = call_intrin("float32", "tir.sqrt", tvm.tir.FloatImm("float32", 2.0))
showmod(x)

x = call_intrin("float32", "tir.fabs", tvm.tir.FloatImm("float32", 2.0))
showmod(x)

x = call_intrin(
    "float32",
    "tir.bitwise_and",
    tvm.tir.FloatImm("float32", 2.0),
    tvm.tir.FloatImm("float32", 3.0),
)
showmod(x)

x = call_intrin(
    "float32",
    "tir.fma",
    tvm.tir.FloatImm("float32", 2.0),
    tvm.tir.FloatImm("float32", 3.0),
    tvm.tir.FloatImm("float32", 4.0),
)
showmod(x)

#### call_pure_extern

In [ ]:
x = call_pure_extern("float32", "a_extern_func", x, y)
showmod(x)

#### call_extern

In [ ]:
x = call_extern("float32", "a_extern_func", x, y)
showmod(x)

#### call_llvm_intrin

In [ ]:
x = call_llvm_intrin("float32", "llvm.x86.avx512.sqrt.ps.512", 1)
showmod(x)

#### call_llvm_pure_intrin

In [ ]:
x = call_llvm_pure_intrin("float32", "llvm.x86.avx512.sqrt.ps.512", 1)
showmod(x)

#### tvm_check_return

In [ ]:
x = tvm_check_return(1, 0, tvm.tir.IntImm("int32", 1))
showmod(x)

In [ ]:
# TODO

#### 